In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy
import shapely
import fiona
import geopandas as gpd
import pysal
import cartopy
import mapclassify
import rasterio
import rasterstats
import earthpy
import geoplot
import geoviews
import folium
try:
    import gdal
except ImportError:
    from osgeo import gdal

c:\Users\aghil\anaconda3\envs\geopy\lib\site-packages\geoviews\operation\__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.18, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (


In [2]:
# Import necessary geometric objects from shapely module
from shapely.geometry import Point, LineString, Polygon

# Create Point geometric object(s) with coordinates
point1 = Point(2.2, 4.2)
point2 = Point(7.2, -25.1)
point3 = Point(9.26, -2.456)

point3D = Point(9.26, -2.456, 0.57)

# What is the type of the point?
point_type = type(point1)

# Print point information
print(point1)

print(point3D)

print(type(point1))

POINT (2.2 4.2)
POINT Z (9.26 -2.456 0.57)
<class 'shapely.geometry.point.Point'>


In [3]:
# Get only x coordinates of Point1
x = point1.x
print(x)

# Get only y coordinates of Point1
y = point1.y
print(y)

# Get x and y coordinates
point_coords = point1.coords
print(type(point_coords))

xy = point_coords.xy
print(xy)

2.2
4.2
<class 'shapely.coords.CoordinateSequence'>
(array('d', [2.2]), array('d', [4.2]))


In [4]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

# Origin point
orig = Point(0, 0)

# Destination points
dest1 = Point(0, 1.45)
dest2 = Point(2, 2)
dest3 = Point(0, 2.5)
dest4 = Point(1.45, 0)

destinations = MultiPoint([dest4, dest2, dest3, dest1])
print(destinations)

# Processing A nearest point from orig
nearest_geoms = nearest_points(orig, destinations)
print( type(nearest_geoms) )

print(nearest_geoms)
print(nearest_geoms[0])
print(nearest_geoms[1])

MULTIPOINT (1.45 0, 2 2, 0 2.5, 0 1.45)
<class 'tuple'>
(<shapely.geometry.point.Point object at 0x0000028A1DE0BD00>, <shapely.geometry.point.Point object at 0x0000028A23031BB0>)
POINT (0 0)
POINT (1.45 0)


In [5]:
# Create a LineString from our Point objects
line = LineString([point1, point2, point3])
print(line)
print( type(line) )

# It is also possible to use coordinate tuples having the same outcome
line2 = LineString([(2.2, 4.2), (7.2, -25.1), (9.26, -2.456)])
print(line2)

# Get x and y coordinates of the line (an array of array of double, where line.xy[0] correspond to abscisses and line.xy[0] to ordinates)
lxy = line.xy
print(lxy)

# Extract x coordinates
line_x = lxy[0]
print(line_x)

# Extract y coordinates 
line_y = line.xy[1]
print(line_y)

# Get the length of the line
l_length = line.length
print("Length of our line: {0:.2f}".format(l_length))

# Get the centroid of the line
l_centroid = line.centroid
print("Centroid of our line: ", l_centroid)

# What type is the centroid? A point
centroid_type = type(l_centroid)
print("Type of the centroid:", centroid_type)

LINESTRING (2.2 4.2, 7.2 -25.1, 9.26 -2.456)
<class 'shapely.geometry.linestring.LineString'>
LINESTRING (2.2 4.2, 7.2 -25.1, 9.26 -2.456)
(array('d', [2.2, 7.2, 9.26]), array('d', [4.2, -25.1, -2.456]))
array('d', [2.2, 7.2, 9.26])
array('d', [4.2, -25.1, -2.456])
Length of our line: 52.46
Centroid of our line:  POINT (6.229961354035622 -11.89241115757239)
Type of the centroid: <class 'shapely.geometry.point.Point'>


In [6]:
from shapely.geometry import LineString, MultiLineString

# Create two lines
line_a = LineString([(0, 0), (1, 1)])
line_b = LineString([(1, 1), (0, 2)])

# Check if lines intersects
lines_intersect = line_a.intersects(line_b)
print("Do lines intersect ? ",lines_intersect)

# Check if lines touches
lines_touche = line_a.touches(line_b)
print("Do lines touch ? ",lines_touche)

Do lines intersect ?  True
Do lines touch ?  True


In [7]:
# Create a Polygon from the coordinates
poly = Polygon([(2.2, 4.2), (7.2, -25.1), (9.26, -2.456)])
print(poly)
print( type(poly) )

# Create a Polygon from the points
poly2 = Polygon([point1, point2, point3])
print(poly2)

# We can also use our previously created Point objects (same outcome)
# --> notice that Polygon object requires x,y coordinates as input
poly3 = Polygon([[p.x, p.y] for p in [point1, point2, point3]])
print(poly3)

# Get the centroid of the Polygon
poly_centroid = poly.centroid
print("Poly centroid: ", poly)

# Get the area of the Polygon
poly_area = poly.area
print("Poly Area: ",poly_area)

# Get the bounds of the Polygon (i.e. bounding box)
poly_bbox = poly.bounds
print("Poly Bounding Box: ",poly)

# Get the exterior of the Polygon
poly_ext = poly.exterior
print("Poly Exterior: ", poly_ext)

# Get the length of the exterior
poly_ext_length = poly_ext.length
print("Poly Exterior Length: ", poly_ext_length)

POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
<class 'shapely.geometry.polygon.Polygon'>
POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
Poly centroid:  POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
Poly Area:  86.789
Poly Bounding Box:  POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
Poly Exterior:  LINEARRING (2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2)
Poly Exterior Length:  62.16395199996553


In [8]:
from shapely.geometry import Point, Polygon

# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

# Create a Polygon
coords = [
    (24.950899, 60.169158),
    (24.953492, 60.169158),
    (24.953510, 60.170104),
    (24.950958, 60.169990),
]
poly = Polygon(coords)

# Let's check what we have
print(p1)
print(p2)
print(poly)

# Check if p1 is within the polygon using the within function
p1_within = p1.within(poly)
print("p1 is within the polygon :", p1_within)

# Does polygon contain p1?
contains_p1 = poly.contains(p1)
print("poly contains p1 :", contains_p1)

# Check if p2 is within the polygon
p2_within = p2.within(poly)
print("p2 is within the polygon :", p2_within)

# Does polygon contain p2?
contains_p2 = poly.contains(p2)
print("poly contains p2 :", contains_p2)

POINT (24.952242 60.1696017)
POINT (24.976567 60.16125)
POLYGON ((24.950899 60.169158, 24.953492 60.169158, 24.95351 60.170104, 24.950958 60.16999, 24.950899 60.169158))
p1 is within the polygon : True
poly contains p1 : True
p2 is within the polygon : False
poly contains p2 : False


In [9]:
from shapely.geometry import MultiPoint, MultiLineString, MultiPolygon

# Create a collection of two points
multipoint = MultiPoint([Point(2, 2), Point(3, 3)])
print(multipoint)

# Create a collection of two lines
multiline = MultiLineString(
    [LineString([(2, 2), (3, 3)]), LineString([(4, 3), (6, 4)])]
)
print(multiline)

# Create a collection of two polygons
multipoly = MultiPolygon(
    [Polygon([(0, 0), (0, 4), (4, 4)]), Polygon([(6, 6), (6, 12), (12, 12)])]
)
print(multipoly)

MULTIPOINT (2 2, 3 3)
MULTILINESTRING ((2 2, 3 3), (4 3, 6 4))
MULTIPOLYGON (((0 0, 0 4, 4 4, 0 0)), ((6 6, 6 12, 12 12, 6 6)))


In [10]:
# Convex Hull of our MultiPoint --> https://en.wikipedia.org/wiki/Convex_hull
convex = multipoint.convex_hull
print("Convex hull of the points: ", convex)

# Let's calculate the area of our MultiPolygon
multi_poly_area = multipoly.area
print("Area of our MultiPolygon:", multi_poly_area)

# We can check if we have a "valid" MultiPolygon. MultiPolygon is thought as valid if the individual polygons does not intersect with each other. This can be really useful information when trying to find topological errors from your data
valid = multipoly.is_valid
print("Is polygon valid?: ", valid)

Convex hull of the points:  LINESTRING (2 2, 3 3)
Area of our MultiPolygon: 26.0
Is polygon valid?:  True
